# Importing Libraries

In [13]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import normalize
from sklearn.decomposition import NMF
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
import gensim
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

# Importing data

In [2]:
content_syracuse = pd.read_csv("C:/Users/adity/Downloads/SRC Project/LDA/pub1.csv", engine = 'python')

# Clean headline column

In [3]:
content_syracuse["Headlines"] = content_syracuse["Headlines"].astype('str')
content_syracuse["Headlines"] = content_syracuse["Headlines"].str.encode("ascii", "ignore").str.decode('ascii')

# Create new column for level 1 tags

In [4]:
content_syracuse["level_1"] = content_syracuse['pagePathLevel1']
# remove / from page path
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('/','')
# make blank rows as others
content_syracuse['level_1'].fillna('others', inplace = True)

# Creating bigrams / trigrams

In [5]:
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('new york','ny')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('syracuse university','su')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('president trump','trump')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('donald trump','trump')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('new year','new year')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace("trump's",'trump')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('president obama','obama')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('dear abby','dearabby')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('pm buzz','pmbuzz')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('viral video','viralvideo')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('star wars','starwars')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('white house','whitehouse')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('high school','highschool')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace("women's",'woman')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace("men's",'men')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace("buffalo-bill",'buffalobill')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace("boeheims",'boeheim')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace("boeheim's",'boeheim')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace("jim boeheim",'boeheim')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace("information",'info')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace("notre dame",'notredame')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('syracuse','')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('cny','')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('ny','')

# Cleaning up level 1 tags

In [6]:
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('orangebasketball','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('orangefootball','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('golf','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('orangelacrosse','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('ny-sports','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('bowling','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('buffalo-bills','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('olympics','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('superbowl','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('sports-talk','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('su-sports-talk','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('rec-sports','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('collegefootball','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('highschoolsports','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('orangesports','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('hssports-special','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('collegelacrosse','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('collegebasketball','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('crunch','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('orangewomen','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('marchmadness','sports')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('college-sports','sports')

content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('us-news','news')

content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('business-news','business')

content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('celebrity-news','entertainment')

content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('poliquin','opinion')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('axeman','opinion')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('kirst','opinion')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('borkowski','opinion')

content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('food','life-and-culture')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('drinks','life-and-culture')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('life-and-culturefitness','life-and-culture')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('health','life-and-culture')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('life_and_culture','life-and-culture')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('restaurants','life-and-culture')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('living','life-and-culture')

content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('statefair','localnews')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('state','localnews')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('expo','localnews')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('schools','localnews')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('cny','localnews')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('neighbors','localnews')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('outdoors','localnews')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('realelocalnews-news','localnews')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('zoo','localnews')
content_syracuse['level_1'] = content_syracuse['level_1'].str.replace('localnews-speaks','localnews')

# Creating level 1 tag for obits

In [7]:
for i in range(0, len(content_syracuse)):
    if (re.search("today's obit:", str(content_syracuse.Headlines[i]), re.IGNORECASE)):
        content_syracuse["level_1"][i] = "obituaries"
    elif (re.search("today's obits:", str(content_syracuse.Headlines[i]), re.IGNORECASE)):
        content_syracuse["level_1"][i] = "obituaries"
    elif (re.search("today's obituaries:", str(content_syracuse.Headlines[i]), re.IGNORECASE)):
        content_syracuse["level_1"][i] = "obituaries"

# Create count of headlines per level 1 tag and filter < 1000 as others

In [8]:
content_syracuse['counts'] = content_syracuse['level_1'].map(content_syracuse['level_1'].value_counts())
content_syracuse.loc[content_syracuse.counts < 1000, 'level_1'] = "others"

# Use ML to classify "others" as level 1 tag

# Stemming & Tokenizing

In [9]:
extra_stop_words = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', "%", 'm', 'p', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 
 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "about", "across", "after", "all", 
 "also", "an", "and", "another", "added", "any", "are", "as", "at", "basically", "be", "because", 'become', "been", "before", 
 "being", "between", "both", "but", "by", "came", "can", "come", "could", "did", "do", "does", "each", "else", "every",
 "either", "especially", "for", "from", "get", "given", "gets", 'give', 'gives', "got", "goes", "had", "has", "have", "he", 
 "her", "here", "him", "himself", "his", "how", "if", "in", "into", "is", "it", "its", "just", "lands", "like", "make", 
 "making", "made", "many", "may", "me", "might", "more", "most", "much", "must", "my", "never", "provide", "provides", 
 "perhaps", "no", "now", "of", "on", "only", "or", "other", "our", "out", "over", "re", "said", "same", "see", "should", 
 "since", "so", "some", "still", "such", "seeing", "see", "take", "than", "that", "the", "their", "them", "then", "there", 
 "these", "they", "this", "those", "through", "to", "too", "under", "up", "use", "using", "used", "underway", "very", "want", 
 "was", "way", "we", "well", "were", "what", "when", "where", "which", "while", "whilst", "who", "will", "with", "would", 
 "you", "your", 'etc', 'via', 'eg', 'news', "'s", "april", "march", "june", "july", "august", "february", "january", "december", 
 "september", "october", "dec", "oct", "nov", "today's", "'the'", "sept", "feb", "jan", "amp", "say", "'the", "friday's", "friday",
 "saturday", "saturday's", "sunday", "sunday's"]
stop_words = text.ENGLISH_STOP_WORDS.union(extra_stop_words)

content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('obituaries','obits')
content_syracuse['Headlines'] = content_syracuse['Headlines'].str.replace('obituary','obits')

stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')
wn = WordNetLemmatizer()

def tokenize(text):
    return [wn.lemmatize(word) for word in tokenizer.tokenize(text.lower())]

# Creating subset without "others"

In [10]:
content_subset_without_others = content_syracuse[content_syracuse["level_1"] != "others"]
content_syracuse["pred"] = content_syracuse["level_1"]
content_syracuse["conf"] = 0
content_syracuse["pred"].value_counts()

news                81587
others              33531
sports              33307
entertainment       23353
localnews           21659
crime               15204
opinion             14184
life-and-culture    11237
business             7108
politics             4881
weather              2363
obituaries           1150
Name: pred, dtype: int64

In [11]:
content_subset_without_others["level_1"].value_counts()

news                81587
sports              33307
entertainment       23353
localnews           21659
crime               15204
opinion             14184
life-and-culture    11237
business             7108
politics             4881
weather              2363
obituaries           1150
Name: level_1, dtype: int64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(content_subset_without_others['Headlines'], content_subset_without_others['level_1'], random_state = 0)
count_vect = CountVectorizer(min_df = 30, stop_words = stop_words, tokenizer = tokenize, analyzer = 'word', max_features = 50000)
X_train_counts = count_vect.fit_transform(X_train)
tfidf = TfidfTransformer(sublinear_tf = True, norm = 'l2', smooth_idf = False)
X_train_tfidf = tfidf.fit_transform(X_train_counts)
clf = CalibratedClassifierCV(LinearSVC()).fit(X_train_tfidf, y_train)

In [15]:
content_syracuse["pred"] = clf.predict(count_vect.transform(content_syracuse["Headlines"]))
content_syracuse["conf"] = (clf.predict_proba(count_vect.transform(content_syracuse["Headlines"]))).max(axis = 1)
content_syracuse["pred"].value_counts()

news                69243
sports              45382
crime               33545
entertainment       29783
localnews           22534
life-and-culture    13350
business            11582
opinion             10500
politics             7798
weather              4656
obituaries           1191
Name: pred, dtype: int64

In [16]:
content_syracuse["final_pred"] = np.where((content_syracuse["level_1"] == "others") & (content_syracuse["conf"] >= 0.6), content_syracuse["pred"], content_syracuse["level_1"])
content_syracuse['counts'] = content_syracuse['final_pred'].map(content_syracuse['final_pred'].value_counts())
content_syracuse.to_csv("content_syracuse.csv")

In [17]:
content_syracuse["final_pred"].value_counts()

news                85283
sports              36815
entertainment       26192
localnews           22921
others              20202
crime               15595
opinion             14659
life-and-culture    11764
business             7455
politics             5086
weather              2441
obituaries           1151
Name: final_pred, dtype: int64

# Subsetting each of the tags

In [18]:
content_subset_obit = content_syracuse[content_syracuse["final_pred"] == "obituaries"]
content_subset_weather = content_syracuse[content_syracuse["final_pred"] == "weather"]
content_subset_politics = content_syracuse[content_syracuse["final_pred"] == "politics"]
content_subset_business = content_syracuse[content_syracuse["final_pred"] == "business"]
content_subset_landc = content_syracuse[content_syracuse["final_pred"] == "life-and-culture"]
content_subset_opinion = content_syracuse[content_syracuse["final_pred"] == "opinion"]
content_subset_crime = content_syracuse[content_syracuse["final_pred"] == "crime"]
content_subset_localnews = content_syracuse[content_syracuse["final_pred"] == "localnews"]
content_subset_entertainment = content_syracuse[content_syracuse["final_pred"] == "entertainment"]
content_subset_sports = content_syracuse[content_syracuse["final_pred"] == "sports"]
content_subset_news = content_syracuse[content_syracuse["final_pred"] == "news"]
content_subset_others = content_syracuse[content_syracuse["final_pred"] == "others"]

# Methods to try
1. K Means
2. Guided LDA
3. LDA
4. NMF

# K Means

## Obits

In [19]:
content_subset_headline = content_subset_obit[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 3
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : wa, obit, teacher, year, school, highschool, john, james, north
Cluster 1 : worked, obit, year, school, james, jr, graduated, michael, highschool
Cluster 2 : obit, graduated, highschool, year, owned, school, retired, served, jr


## Weather

In [20]:
content_subset_headline = content_subset_weather[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 4
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : upstate, storm, rain, week, heat, winter, today, severe, wind
Cluster 1 : snow, effect, lake, storm, upstate, central, inch, winter, foot
Cluster 2 : weather, central, weekend, week, upstate, winter, cold, day, start
Cluster 3 : central, storm, day, school, winter, record, county, wind, year


## Politics

In [21]:
content_subset_headline = content_subset_politics[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 5
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : race, senate, cuomo, democratic, state, congress, schumer, debate, democrat
Cluster 1 : john, katko, rep, balter, dana, congress, house, video, trump
Cluster 2 : trump, border, whitehouse, video, campaign, president, democrat, wall, report
Cluster 3 : election, county, onondaga, day, ballot, voter, win, primary, result
Cluster 4 : gop, trump, tax, plan, candidate, rep, primary, senate, house


## Business

In [22]:
content_subset_headline = content_subset_business[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 4
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : new, compa, named, store, open, close, job, dewitt, building
Cluster 1 : joined, compa, group, associate, terakeet, st, pinckney, hugo, barton
Cluster 2 : onondaga, county, judgment, department, inspection, bankruptcy, health, certificate, restaurant
Cluster 3 : promoted, compa, ha, bank, group, cxtec, loguidice, barton, dannible


## Life and Culture

In [23]:
content_subset_headline = content_subset_landc[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 5
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : inspiration, daily, god, love, community, life, change, good, new
Cluster 1 : new, upstate, open, beer, dearabby, video, buzz, restaurant, dinner
Cluster 2 : review, dining, food, fair, offer, restaurant, wine, good, italian
Cluster 3 : best, central, restaurant, pizza, upstate, hunt, chicken, burger, winner
Cluster 4 : photo, prom, day, video, highschool, essay, senior, annual, new


## Opinion

In [24]:
content_subset_headline = content_subset_opinion[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 5
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : editorial, year, new, day, poliquin, ha, state, time, su
Cluster 1 : letter, school, onondaga, county, thank, wa, trump, state, day
Cluster 2 : football, basketball, recap, su, brent, axe, orange, coach, team
Cluster 3 : commentary, trump, need, health, tax, reform, day, people, onondaga
Cluster 4 : cartoon, editorial, trump, mueller, shooting, wall, border, facebook, climate


## Crime

In [25]:
content_subset_headline = content_subset_crime[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 7
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : county, oswego, onondaga, man, deputy, oneida, cayuga, charged, madison
Cluster 1 : judge, murder, court, lawyer, case, trial, plea, accused, prison
Cluster 2 : man, charged, woman, shooting, shot, firefighter, cop, home, house
Cluster 3 : crash, killed, car, county, driver, fatal, police, man, injured
Cluster 4 : year, old, man, police, girl, boy, killed, prison, woman
Cluster 5 : police, man, investigate, shooting, shot, woman, officer, arrest, state
Cluster 6 : murder, accused, trial, man, neulander, dr, suspect, da, case


## Entertainment

In [26]:
content_subset_headline = content_subset_entertainment[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 6
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : best, bet, saturday, saturday's, friday, sunday's, sunday, concert, friday's
Cluster 1 : star, video, music, day, buzz, festival, dy, photo, dead
Cluster 2 : new, movie, buzz, tv, music, star, song, video, trailer
Cluster 3 : concert, tour, date, upstate, lakeview, amphitheater, include, buffalo, calendar
Cluster 4 : pmbuzz, trailer, buzz, movie, new, tv, star, reboot, arrested
Cluster 5 : review, playhouse, theatre, stage, cortland, rarely, chevy, fun, fair


## Local news

In [27]:
content_subset_headline = content_subset_localnews[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 7
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : lake, new, state, photo, basketball, fishing, man, year, buzz
Cluster 1 : prom, photo, junior, highschool, senior, ball, central, academy, dance
Cluster 2 : upstate, police, man, ranked, best, woman, school, photo, year
Cluster 3 : fair, food, review, state, row, chevy, court, day, building
Cluster 4 : house, week, home, colonial, road, st, cazenovia, manlius, ha
Cluster 5 : graduation, highschool, class, list, valedictorian, salutatorian, central, candidate, sr
Cluster 6 : school, student, highschool, musical, present, spring, perform, district, fall


## Sports

In [28]:
content_subset_headline = content_subset_sports[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 8
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : crunch, buffalo, team, nfl, new, coach, video, nba, tournament
Cluster 1 : basketball, woman, tournament, team, ncaa, game, orange, acc, college
Cluster 2 : player, basketball, football, su, lacrosse, nba, woman, team, year
Cluster 3 : win, game, crunch, title, championship, basketball, state, speedway, video
Cluster 4 : football, recruiting, su, team, coach, star, qb, game, dino
Cluster 5 : lacrosse, men, woman, team, su, boy, girl, win, ncaa
Cluster 6 : jim, boeheim, coach, basketball, boeheim's, conference, boeheims, watch, win
Cluster 7 : channel, tv, stream, live, time, info, information, game, tipoff


## News

In [29]:
content_subset_headline = content_subset_news[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 8
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : crash, killed, car, fatal, man, injured, driver, woman, update
Cluster 1 : state, fair, class, championship, girl, boy, win, semifinal, ranking
Cluster 2 : police, man, officer, shooting, woman, arrest, charge, identify, chief
Cluster 3 : man, charged, accused, year, charge, prison, sentenced, county, oswego
Cluster 4 : win, lottery, number, million, jackpot, winning, powerball, mega, girl
Cluster 5 : county, onondaga, oswego, lake, cayuga, madison, sheriff, deputy, judgment
Cluster 6 : new, school, year, today, compa, central, highschool, su, student
Cluster 7 : section, iii, boy, girl, class, basketball, star, state, championship


## Others

In [30]:
content_subset_headline = content_subset_others[['Headlines']]
document = list(content_subset_headline["Headlines"])
vectorizer = TfidfVectorizer(sublinear_tf = True, min_df = 30, norm = 'l2', stop_words = stop_words, tokenizer = tokenize)
X = vectorizer.fit_transform(document)
true_k = 8
model = KMeans(n_clusters = true_k, init = 'k-means++', max_iter = 500, n_init = 20)
model.fit(X)
terms = vectorizer.get_feature_names()
words = model.cluster_centers_.argsort()[:,-1:-10:-1]
for num, centroid in enumerate(words):
    print('Cluster ' + str(num) + ' : ' + ', '.join(terms[word] for word in centroid))

Cluster 0 : photo, central, week, chief, video, upstate, state, best, time
Cluster 1 : su, student, video, sport, great, time, alum, campus, frat
Cluster 2 : new, desti, usa, store, open, mall, tv, photo, year
Cluster 3 : year, county, onondaga, old, madison, oswego, lake, state, cayuga
Cluster 4 : ski, report, th, rd, nd, saturday, friday, claim, ticket
Cluster 5 : war, memorial, live, john, james, brown, onondaga, animal, lady
Cluster 6 : day, home, parade, viralvideo, custom, builder, photo, summercuse, house
Cluster 7 : man, crash, police, traffic, alert, interstate, update, charged, lane


# Guided LDA

In [36]:
# simplify Penn tags to n (NOUN), v (VERB), a (ADJECTIVE) or r (ADVERB)
def simplify(penn_tag):
    pre = penn_tag[0]
    if (pre == 'J'):
        return 'a'
    elif (pre == 'R'):
        return 'r'
    elif (pre == 'V'):
        return 'v'
    else:
        return 'n'
    
def preprocess(text):
    toks = gensim.utils.simple_preprocess(str(text), deacc = True)
    wn = WordNetLemmatizer()
    return [wn.lemmatize(tok, simplify(pos)) for tok, pos in nltk.pos_tag(toks) if tok not in stop_words]

def test_eta_lda(eta, dictionary, ntopics, print_topics = True, print_dist = True):
    np.random.seed(42) # set the random seed for repeatability
    bow = [dictionary.doc2bow(line) for line in corp] # get the bow-format lines with the set dictionary
    with (np.errstate(divide = 'ignore')):  # ignore divide-by-zero warnings
        model = gensim.models.ldamodel.LdaModel(
            corpus = bow, id2word = dictionary, num_topics = ntopics,
            random_state = 42, eta = eta, alpha = 'auto')
    # visuzlize the model term topics
    print('Perplexity: {:.2f}'.format(model.log_perplexity(bow)))
    if print_topics:
        # display the top terms for each topic
        for topic in range(ntopics):
            print('Topic {}: {}'.format(topic, [dictionary[w] for w, p in model.get_topic_terms(topic, topn = 10)]))
#     if print_dist:
#         # display the topic probabilities for each document
#         for line, bag in zip(txt, bow):
#             doc_topics = ['({}, {:.1%})'.format(topic, prob) for topic, prob in model.get_document_topics(bag)]
#             print('{} {}'.format(line, doc_topics))
    return model

def test_eta_nmf(eta, dictionary, train_headlines_sentences, ntopics, print_topics = True, print_dist = True):
    vectorizer = CountVectorizer(analyzer = 'word', max_features = 50000, min_df = 30, stop_words = stop_words, tokenizer = tokenize)
    x_counts = vectorizer.fit_transform(train_headlines_sentences)
    # we set a TfIdf Transformer, and transform the counts with the model
    transformer = TfidfTransformer(smooth_idf = False, sublinear_tf = True, norm = 'l2')
    x_tfidf = transformer.fit_transform(x_counts)
    # normalize the TfIdf values to unit length for each row
    xtfidf_norm = normalize(x_tfidf, norm = 'l1', axis = 1)
    # obtain an NMF model
    model = NMF(n_components = ntopics, init = 'nndsvd');
    # fit the model
    model.fit(xtfidf_norm)
    if print_topics:
        # display the top terms for each topic
        feat_names = vectorizer.get_feature_names()
        word_dict = {}
        for i in range(ntopics):
            # for each topic, obtain the largest values, and add the words they map to into the dictionary.
            words_ids = model.components_[i].argsort()[:-10 - 1:-1]
            words = [feat_names[key] for key in words_ids]
            word_dict['Topic #' + '{:02d}'.format(i + 1)] = words
        print(pd.DataFrame(word_dict))
    return model

def create_eta(priors, etadict, ntopics):
    eta = np.full(shape = (ntopics, len(etadict)), fill_value = 1) # create a (ntopics, nterms) matrix and fill with 1
    for word, topic in priors.items(): # for each word in the list of priors
        keyindex = [index for index,term in etadict.items() if term == word] # look up the word in the dictionary
        if (len(keyindex) > 0): # if it's in the dictionary
            eta[topic, keyindex[0]] = 1e7  # put a large number in there      
    eta = np.divide(eta, eta.sum(axis = 0)) # normalize so that the probabilities sum to 1 over all topics
    return eta

# LDA and NMF

## Obits

In [37]:
txt = list(content_subset_obit["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 3)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 3)

Perplexity: -6.99
Topic 0: ['today', 'obits', 'year', 'work', 'james', 'obit', 'school', 'dr', 'own', 'teacher']
Topic 1: ['today', 'obits', 'work', 'school', 'year', 'obit', 'teacher', 'north', 'graduate', 'retire']
Topic 2: ['today', 'obits', 'graduate', 'highschool', 'year', 'work', 'jr', 'own', 'john', 'school']
    Topic #01   Topic #02 Topic #03
0        obit        work      year
1       today       today    retire
2    graduate        obit    school
3  highschool      school   teacher
4      school     michael     today
5          jr          jr      obit
6        john       james     north
7       serve   liverpool     james
8     teacher    graduate    county
9          dr  highschool   michael


NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=3, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Weather

In [38]:
txt = list(content_subset_weather["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 4)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 4)

Perplexity: -6.61
Topic 0: ['upstate', 'winter', 'storm', 'snow', 'weather', 'wet', 'forecast', 'warm', 'year', 'week']
Topic 1: ['upstate', 'snow', 'central', 'winter', 'storm', 'video', 'wind', 'county', 'week', 'warm']
Topic 2: ['upstate', 'snow', 'weather', 'storm', 'lake', 'central', 'effect', 'rain', 'winter', 'heat']
Topic 3: ['weather', 'central', 'weekend', 'upstate', 'wind', 'winter', 'snow', 'cold', 'day', 'school']
  Topic #01 Topic #02 Topic #03 Topic #04
0   weather   upstate      snow   central
1      week      week     storm   weekend
2   weekend     storm      lake       day
3    winter    winter    effect      cold
4    spring      rain    winter    winter
5      cool     flood  blizzard    record
6     start     today      inch      warm
7      cold      cold      rain     power
8     close      warm      foot    school
9     photo      wind     video     snowy


NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=4, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Politics

In [39]:
txt = list(content_subset_politics["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 5)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 5)

Perplexity: -7.77
Topic 0: ['trump', 'john', 'katko', 'clinton', 'rep', 'congress', 'say', 'hillary', 'new', 'vote']
Topic 1: ['trump', 'county', 'election', 'primary', 'onondaga', 'race', 'win', 'upstate', 'gop', 'candidate']
Topic 2: ['trump', 'border', 'wall', 'say', 'miner', 'schumer', 'stephanie', 'whitehouse', 'race', 'su']
Topic 3: ['trump', 'say', 'debate', 'republican', 'democratic', 'rally', 'presidential', 'tariff', 'tax', 'woman']
Topic 4: ['trump', 'say', 'rep', 'whitehouse', 'gop', 'john', 'tenney', 'video', 'campaign', 'cut']
     Topic #01   Topic #02 Topic #03     Topic #04 Topic #05
0        trump        john  election        senate     state
1     campaign       katko    county      democrat     cuomo
2   whitehouse         rep       win     president       tax
3        rally    congress  onondaga       schumer     union
4      america    campaign       day           gop  lawmaker
5       report       video     voter           run       gov
6      mueller      balter

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=5, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Business

In [40]:
txt = list(content_subset_business["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 4)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 4)

Perplexity: -7.68
Topic 0: ['county', 'onondaga', 'store', 'close', 'restaurant', 'department', 'bankruptcy', 'new', 'compa', 'open']
Topic 1: ['compa', 'join', 'new', 'hotel', 'inside', 'look', 'central', 'million', 'dewitt', 'photo']
Topic 2: ['county', 'onondaga', 'inspection', 'restaurant', 'tax', 'mall', 'new', 'compa', 'health', 'business']
Topic 3: ['compa', 'join', 'health', 'department', 'name', 'group', 'promote', 'post', 'ford', 'standard']
     Topic #01    Topic #02  Topic #03   Topic #04
0         join       county    promote         new
1        compa     onondaga      compa        open
2    associate   bankruptcy       bank       store
3        group     judgment      cxtec       close
4     terakeet     business      group       hotel
5  engineering  certificate      award  restaurant
6      inficon      warrant  loguidice       photo
7           st          tax     barton      dewitt
8         bank   restaurant    manager       desti
9      michael   inspection   dire

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=4, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Life and Culture

In [41]:
txt = list(content_subset_landc["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 5)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 5)

Perplexity: -8.32
Topic 0: ['photo', 'video', 'day', 'upstate', 'new', 'love', 'inside', 'kid', 'dinner', 'pick']
Topic 1: ['open', 'new', 'restaurant', 'upstate', 'home', 'photo', 'close', 'county', 'bar', 'buzz']
Topic 2: ['review', 'wine', 'beer', 'din', 'food', 'week', 'history', 'taste', 'new', 'video']
Topic 3: ['best', 'central', 'inspiration', 'restaurant', 'hospital', 'upstate', 'medical', 'daily', 'video', 'chicken']
Topic 4: ['photo', 'upstate', 'new', 'week', 'day', 'home', 'chef', 'lake', 'open', 'center']
     Topic #01     Topic #02   Topic #03   Topic #04   Topic #05
0  inspiration        dinner       photo        food        buzz
1        daily    restaurant       video      review         new
2         love          love         day        fair     trailer
3          god       kitchen        prom         din       movie
4    community  fayetteville     upstate  restaurant     central
5         life         house     central         new        star
6        power      

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=5, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Opinion

In [42]:
txt = list(content_subset_opinion["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 5)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 5)

Perplexity: -8.57
Topic 0: ['letter', 'editorial', 'cartoon', 'commentary', 'year', 'im', 'change', 'new', 'community', 'climate']
Topic 1: ['basketball', 'football', 'orange', 'su', 'recap', 'coach', 'jim', 'game', 'boeheim', 'team']
Topic 2: ['letter', 'commentary', 'trump', 'world', 'reader', 'school', 'war', 'stop', 'city', 'need']
Topic 3: ['letter', 'commentary', 'day', 'su', 'voting', 'today', 'thank', 'year', 'world', 'veteran']
Topic 4: ['letter', 'commentary', 'onondaga', 'county', 'editorial', 'school', 'vote', 'state', 'football', 'year']
   Topic #01    Topic #02   Topic #03   Topic #04 Topic #05
0     letter    editorial  commentary    football    strong
1     school      cartoon       trump          su    memory
2      trump        trump        need       recap     woman
3   onondaga   basketball         tax  basketball      good
4        day      mueller        stop      orange     right
5      today  endorsement         day        year      lead
6  community     facebo

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=5, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Crime

In [43]:
txt = list(content_subset_crime["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 7)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 7)

Perplexity: -7.33
Topic 0: ['county', 'crash', 'charge', 'man', 'onondaga', 'deputy', 'sheriff', 'kill', 'oneida', 'oswego']
Topic 1: ['murder', 'trial', 'case', 'year', 'judge', 'man', 'day', 'court', 'death', 'heidi']
Topic 2: ['man', 'charge', 'police', 'murder', 'accuse', 'woman', 'trooper', 'stab', 'girlfriend', 'death']
Topic 3: ['police', 'year', 'man', 'old', 'woman', 'crash', 'kill', 'stab', 'teen', 'car']
Topic 4: ['police', 'man', 'arrest', 'charge', 'woman', 'suspect', 'gun', 'steal', 'investigate', 'video']
Topic 5: ['update', 'crash', 'county', 'student', 'firefighter', 'woman', 'police', 'close', 'charge', 'kill']
Topic 6: ['police', 'man', 'shoot', 'shot', 'witness', 'murder', 'south', 'street', 'hit', 'kill']
     Topic #01 Topic #02 Topic #03 Topic #04 Topic #05  Topic #06    Topic #07
0       police    county       man      year    arrest     murder        shoot
1  investigate     crash    charge       old  homicide      judge  investigate
2        woman    oswego   

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=7, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Entertainment

In [44]:
txt = list(content_subset_entertainment["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 6)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 6)

Perplexity: -8.78
Topic 0: ['new', 'book', 'movie', 'tv', 'theater', 'author', 'band', 'su', 'turn', 'series']
Topic 1: ['buzz', 'trailer', 'voice', 'pmbuzz', 'star', 'day', 'movie', 'new', 'su', 'fair']
Topic 2: ['dead', 'star', 'walk', 'tv', 'buzz', 'actor', 'pmbuzz', 'new', 'death', 'dy']
Topic 3: ['musical', 'thing', 'city', 'singer', 'weekend', 'actor', 'finale', 'dead', 'new', 'band']
Topic 4: ['star', 'pmbuzz', 'round', 'video', 'trailer', 'battle', 'merry', 'upstate', 'review', 'singer']
Topic 5: ['review', 'festival', 'season', 'music', 'best', 'theatre', 'art', 'film', 'concert', 'stage']
     Topic #01  Topic #02 Topic #03 Topic #04 Topic #05 Topic #06
0          din     review  festival     video      star     music
1   restaurant      stage   concert     photo       new     award
2    liverpool  playhouse      best      game     movie      week
3          inn    theatre       bet      miss      dead     scene
4        north   redhouse      fair      song        dy      cus

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=6, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Local news

In [45]:
txt = list(content_subset_localnews["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 7)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 7)

Perplexity: -8.40
Topic 0: ['upstate', 'man', 'woman', 'western', 'police', 'deer', 'kill', 'charge', 'year', 'arrest']
Topic 1: ['west', 'genesee', 'north', 'art', 'new', 'award', 'upstate', 'owner', 'dinner', 'president']
Topic 2: ['school', 'state', 'upstate', 'fair', 'new', 'district', 'best', 'day', 'student', 'central']
Topic 3: ['lake', 'fish', 'angler', 'river', 'catch', 'salmon', 'football', 'oneida', 'big', 'watch']
Topic 4: ['week', 'whats', 'state', 'upstate', 'academy', 'delay', 'store', 'lake', 'fishing', 'eat']
Topic 5: ['highschool', 'prom', 'photo', 'house', 'student', 'junior', 'home', 'senior', 'onondaga', 'week']
Topic 6: ['fair', 'food', 'review', 'highschool', 'graduation', 'court', 'class', 'chevy', 'king', 'upstate']
       Topic #01    Topic #02      Topic #03 Topic #04    Topic #05  \
0           fair         week     highschool   upstate        photo   
1           food        house     graduation      best         prom   
2         review         home       

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=7, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Sports

In [46]:
txt = list(content_subset_sports["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 8)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 8)

Perplexity: -8.11
Topic 0: ['football', 'su', 'coach', 'babers', 'dino', 'battle', 'new', 'mets', 'player', 'tyus']
Topic 1: ['basketball', 'jim', 'boeheim', 'game', 'orange', 'watch', 'performance', 'coach', 'conference', 'player']
Topic 2: ['lacrosse', 'win', 'woman', 'team', 'men', 'world', 'su', 'bowl', 'championship', 'game']
Topic 3: ['crunch', 'basketball', 'orange', 'tampa', 'bay', 'football', 'red', 'game', 'ticket', 'player']
Topic 4: ['crunch', 'win', 'yankee', 'game', 'recruiting', 'basketball', 'star', 'woman', 'video', 'national']
Topic 5: ['channel', 'time', 'live', 'tv', 'basketball', 'stream', 'v', 'game', 'info', 'ncaa']
Topic 6: ['football', 'bill', 'buffalo', 'nfl', 'say', 'coach', 'su', 'week', 'receiver', 'offer']
Topic 7: ['football', 'rate', 'grade', 'lacrosse', 'team', 'basketball', 'state', 'north', 'win', 'carolina']
    Topic #01   Topic #02   Topic #03   Topic #04 Topic #05   Topic #06  \
0  basketball    football    lacrosse      crunch      game        te

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=8, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## News

In [47]:
txt = list(content_subset_news["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 8)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 8)

Perplexity: -8.67
Topic 0: ['comment', 'school', 'today', 'new', 'weather', 'student', 'work', 'city', 'law', 'turn']
Topic 1: ['number', 'worker', 'snow', 'start', 'su', 'know', 'wednesday', 'morning', 'good', 'thursday']
Topic 2: ['west', 'genesee', 'girl', 'win', 'section', 'iii', 'basketball', 'class', 'video', 'team']
Topic 3: ['county', 'onondaga', 'state', 'plan', 'lake', 'oswego', 'oneida', 'vote', 'union', 'go']
Topic 4: ['update', 'win', 'state', 'lottery', 'westhill', 'fatal', 'week', 'identify', 'million', 'championship']
Topic 5: ['upstate', 'obits', 'watch', 'whats', 'update', 'center', 'state', 'college', 'national', 'community']
Topic 6: ['man', 'police', 'woman', 'charge', 'year', 'crash', 'kill', 'car', 'old', 'accuse']
Topic 7: ['say', 'court', 'officer', 'traffic', 'police', 'case', 'son', 'trump', 'chief', 'stop']
     Topic #01 Topic #02   Topic #03  Topic #04  Topic #05 Topic #06  \
0       county       man       state      compa        win    school   
1     ono

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=8, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Others

In [48]:
txt = list(content_subset_others["Headlines"])
corp = [preprocess(line) for line in txt]
dictionary = gensim.corpora.Dictionary(corp)
test_eta_lda('auto', dictionary, ntopics = 8)

train_headlines_sentences = [' '.join(text) for text in corp]
test_eta_nmf('auto', dictionary, train_headlines_sentences, ntopics = 8)

Perplexity: -9.37
Topic 0: ['photo', 'prom', 'crash', 'highschool', 'north', 'watch', 'senior', 'dont', 'story', 'upstate']
Topic 1: ['live', 'year', 'help', 'man', 'win', 'charge', 'old', 'teen', 'whats', 'family']
Topic 2: ['upstate', 'day', 'weekend', 'tv', 'life', 'update', 'video', 'ice', 'let', 'event']
Topic 3: ['store', 'new', 'vintage', 'open', 'episode', 'year', 'center', 'desti', 'mall', 'photo']
Topic 4: ['traffic', 'buzz', 'video', 'dy', 'time', 'alert', 'su', 'wife', 'daily', 'oz']
Topic 5: ['county', 'world', 'onondaga', 'chief', 'video', 'new', 'day', 'music', 'real', 'photo']
Topic 6: ['thursday', 'report', 'week', 'series', 'season', 'photo', 'house', 'contest', 'winner', 'race']
Topic 7: ['viralvideo', 'home', 'day', 'fall', 'su', 'turn', 'parade', 'student', 'west', 'photo']
     Topic #01 Topic #02 Topic #03 Topic #04   Topic #05 Topic #06 Topic #07  \
0         week      hole       ski      home       photo       new   central   
1        house       aug    report

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=8, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

# Guided LDA

In [ ]:
apriori_opposite = {
    'crash':0, 'vehicle':0, 'car':0, 'truck':0, 'unlicensed':0, 'pile':0, 'hit':0, 'driver':0,
    'drugs':3, 'heroin':3, 'fentanyl':3, 'street':3,
    'shooting':4, 'gun':4, 'shooter':4, 'shot':4, 'accidental':4, 'armed':4, 'gunpoint':4, 'shoot':4, 'homicide':4,
    'stabbed':5, 'stealing':5, 'murder':5, 'theft':5, 'victim':5, 'suspect':5, 'fatal':5, 'steal':5, 'stab':5,
    'fire':2, 'blaze':2, 'firefighter':2, 'house':2,
    'rape':1, 'child':1, 'abuse':1, 'pornography':1, 'teen':1, 'woman':1,
}
eta = create_eta(apriori_opposite, dictionary, 6)
test_eta_lda(eta, dictionary, 6)